In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00507/wisdm-dataset.zip

--2020-06-21 14:06:04--  https://archive.ics.uci.edu/ml/machine-learning-databases/00507/wisdm-dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 309713877 (295M) [application/x-httpd-php]
Saving to: ‘wisdm-dataset.zip’

100%[======================================>] 309,713,877 88.1MB/s   in 3.6s   

2020-06-21 14:06:09 (82.2 MB/s) - ‘wisdm-dataset.zip’ saved [309713877/309713877]



In [4]:
! unzip wisdm-dataset.zip

Archive:  wisdm-dataset.zip
   creating: wisdm-dataset/
  inflating: wisdm-dataset/WISDM-dataset-description.pdf  
   creating: wisdm-dataset/arffmagic-master/
  inflating: wisdm-dataset/arffmagic-master/Makefile  
  inflating: wisdm-dataset/arffmagic-master/.DS_Store  
 extracting: wisdm-dataset/arffmagic-master/README.md  
   creating: wisdm-dataset/arffmagic-master/src/
  inflating: wisdm-dataset/arffmagic-master/src/arff.cpp  
  inflating: wisdm-dataset/arffmagic-master/src/comparator.h  
  inflating: wisdm-dataset/arffmagic-master/src/chunk.h  
  inflating: wisdm-dataset/arffmagic-master/src/main.cpp  
  inflating: wisdm-dataset/arffmagic-master/src/attribute.h  
  inflating: wisdm-dataset/arffmagic-master/src/libmfcc.c  
  inflating: wisdm-dataset/arffmagic-master/src/raw.h  
  inflating: wisdm-dataset/arffmagic-master/src/try.h  
  inflating: wisdm-dataset/arffmagic-master/src/write.h  
  inflating: wisdm-dataset/arffmagic-master/src/chunk.cpp  
  inflating: wisdm-dataset/arffma

In [5]:
! ls wisdm-dataset/raw

phone  watch


In [6]:
import os

In [7]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8MB 144kB/s  eta 0:00:01
     |████████████████████████████████| 204kB 40.8MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [9]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [10]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

In [11]:
#OUR SCHEMA IS: subjectid,target,timestamp,x,y,z
from pyspark.sql.types import StructType, StructField, IntegerType,StringType,DoubleType
schema = StructType([
    StructField("subjectid",IntegerType(),True),
    StructField("activity",StringType(),True),
    StructField("timestamp",DoubleType(),True),
    StructField("x",DoubleType(),True),
    StructField("y",DoubleType(),True),
    StructField("z",StringType(),True)])#sets z value to string because last char is a ';', will remove in pandas later.


In [13]:
imported_df = None

from pyspark.sql.functions import lit #allows strings/literal to apache spark dataframes
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import expr

raw_path = 'wisdm-dataset/raw/'
file_list_filtered = ['phone','watch']
devices = ['accel','gyro']

for category in file_list_filtered:
    for device in devices:
        path = raw_path+category+'/'+device
        data_files = os.listdir(path)
        
        for data_file in data_files:
            print(path+'/'+data_file)
            temp_df = spark.read.csv(path+'/'+data_file, schema=schema)
            temp_df = temp_df.withColumn('type',lit(category))
            temp_df = temp_df.withColumn('device',lit(device))
            
            if imported_df is None:
                imported_df = temp_df
            else:
                imported_df = imported_df.union(temp_df)
imported_df.printSchema()

wisdm-dataset/raw/phone/accel/data_1627_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1646_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1635_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1616_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1629_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1623_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1606_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1609_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1630_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1620_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1639_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1648_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1615_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1602_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1643_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1622_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1650_accel_phone.txt
wisdm-dataset/raw/phone/accel/data_1604_accel_ph

In [15]:
df = imported_df.withColumn('z', regexp_replace('z', ';', '')) #removes ';' from imported z column
df.withColumn("z", expr("CAST(z AS DOUBLE)"))

df.show()

+---------+--------+-------------------+------------+----------+----------+-----+------+
|subjectid|activity|          timestamp|           x|         y|         z| type|device|
+---------+--------+-------------------+------------+----------+----------+-----+------+
|     1627|       A|4.42607245307544E14|   4.5457916| -8.881348| -9.197556|phone| accel|
|     1627|       A|4.42607265449146E14|-0.114730835|-12.393082| -8.104095|phone| accel|
|     1627|       A|4.42607285590747E14|    1.510315|-13.205139| -6.772949|phone| accel|
|     1627|       A|4.42607305732349E14|   2.7474518|-11.763794|-4.9213257|phone| accel|
|     1627|       A|4.42607325873951E14|    3.040924| -9.194626|-5.7395935|phone| accel|
|     1627|       A|4.42607346015552E14|   4.1273346| -9.052307|-5.5689545|phone| accel|
|     1627|       A|4.42607366157154E14|    4.469742| -9.601379| -5.090271|phone| accel|
|     1627|       A|4.42607386298755E14|   2.2313538| -9.543732|-3.5087433|phone| accel|
|     1627|       A|4

In [16]:
df = df.repartition(1)
df.write.parquet('classifyactivities.parquet')

In [17]:
ls

classifyactivities.parquet/  wisdm-dataset/  wisdm-dataset.zip


In [18]:
ls classifyactivities.parquet/

part-00000-f963ecf1-3b5a-417b-99e3-2e540e9372d1-c000.snappy.parquet  _SUCCESS
